In [18]:
# !pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

## Role Playing, Focus and Cooperation

In [ ]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at Niva Bupa Insurance Agency and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

In [8]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at Niva Bupa Insurance Agency and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

In [9]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

/Users/jayeshbhadane/Desktop/Vettura_AI/GenAI/lib/python3.12/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Users/jayeshbhadane/Desktop/Vettura_AI/GenAI/lib/python3.12/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
/Users/jayeshbhadane/Desktop/Vettura_AI/GenAI/lib/python3.12/site-packages/crewai_tools/tools/selen

In [10]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://www.nivabupa.com/travel-insurance.html"
)

In [ ]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [12]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


# Creating the Crew

In [15]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  memory=True
)

2025-08-09 11:21:40,538 - 4582254080 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [16]:
inputs = {
    "customer": "XYZ Insurance",
    "person": "Jayesh Bhadane",
    "inquiry": "Hey I am travelling to San Francisco for a conference from 22nd to 27 February. How can I get a travel insurance? Can I cancel if my trip gets cancelled?"
}
result = crew.kickoff(inputs=inputs)

# Agent: Senior Support Representative
## Task: XYZ Insurance just reached out with a super important ask:
Hey I am travelling to San Francisco for a conference from 22nd to 27 February. How can I get a travel insurance? Can I cancel if my trip gets cancelled?

Jayesh Bhadane from XYZ Insurance is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


# Agent: Senior Support Representative
## Thought: I need to gather information about travel insurance options and cancellation policies from the Niva Bupa website to provide Jayesh with a comprehensive response.
## Using tool: Read website content
## Tool Input: 
"{}"
## Tool Output: 
Buy Travel Insurance Policy Online in India 2025 @ ₹20/Day* Plans Claims Renewals Resources Customer Support Become an Agent For a New Policy 1800-309-3333 Plans Health Insurance For Me For Family For NRIs For Senior Citizens

/Users/jayeshbhadane/Desktop/Vettura_AI/GenAI/lib/python3.12/site-packages/fireworks/client/image_api.py:131: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("image_strength", pre=True, always=True)
/Users/jayeshbhadane/Desktop/Vettura_AI/GenAI/lib/python3.12/site-packages/fireworks/client/image_api.py:139: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("step_schedule_start", "step_schedule_end", pre=True, alw

# Agent: Support Quality Assurance Specialist
## Task: Review the response drafted by the Senior Support Representative for XYZ Insurance's inquiry. Ensure that the answer is comprehensive, accurate, and adheres to the high-quality standards expected for customer support.
Verify that all parts of the customer's inquiry have been addressed thoroughly, with a helpful and friendly tone.
Check for references and sources used to  find the information, ensuring the response is well-supported and leaves no questions unanswered.


# Agent: Support Quality Assurance Specialist
## Final Answer: 
Hello Jayesh,

Thank you for reaching out! I'm happy to assist you with getting travel insurance for your upcoming trip to San Francisco from February 22nd to 27th.

To obtain travel insurance from Niva Bupa, you can easily purchase a policy directly from their website. Here’s how you can get started:

1. **Visit the Niva Bupa Travel Insurance page**: Go to [Niva Bupa Travel Insurance](https://www.nivabu

- Display the final result as Markdown.

In [ ]:
from IPython.displasy import Markdown
Markdown(result.raw)

Hello Jayesh,

Thank you for reaching out! I'm happy to assist you with getting travel insurance for your upcoming trip to San Francisco from February 22nd to 27th.

To obtain travel insurance from Niva Bupa, you can easily purchase a policy directly from their website. Here’s how you can get started:

1. **Visit the Niva Bupa Travel Insurance page**: Go to [Niva Bupa Travel Insurance](https://www.nivabupa.com/travel-insurance.html).

2. **Fill in your trip details**: You will need to provide details about your travel destination (San Francisco), departure and return dates (February 22nd to 27th), and the number of travelers.

3. **Choose a plan**: Niva Bupa offers different travel insurance plans tailored to suit various needs. You can select from individual, family, or even student travel policies, depending on your preferences.

4. **Complete the purchase**: Once you have selected the appropriate plan, fill in your details, and the payment will complete your purchase.

Regarding your question about trip cancellation: Yes, Niva Bupa’s travel insurance policy includes coverage for trip cancellation. This coverage helps reimburse you for financial losses incurred due to the cancellation of your trip before its onset, allowing you to have peace of mind.

In addition to trip cancellation, Niva Bupa also provides coverage for other important aspects such as medical emergencies, lost luggage, and travel delays, ensuring that you are protected throughout your trip.

If you require further assistance or have any additional questions, feel free to ask. Safe travels, and I hope you have a great conference in San Francisco!

Best regards,  
[Your Name]  
Niva Bupa Support Team